# Test that file handler can download things and use the downloaded copy before downloading again

In [1]:
import sys
sys.path.append('../')

import helpers.fileHandler as fh
import helpers.esgfClient as esgfClient

In [2]:
esgfClient.esgfDownloader('NorESM2-LM', 'psl_Amon', 'piControl', 'r1i1p1f1')

NorESM2-LMpsl_AmonpiControlr1i1p1f1 file not found on ESGF
CMIP6.CMIP.NCC.NorESM2-LM.piControl.r1i1p1f1.Amon.psl.gn.v20210118|noresg.nird.sigma2.no downloading
CMIP6.CMIP.NCC.NorESM2-LM.piControl.r1i1p1f1.Amon.psl.gn.v20210118|esgf-data3.diasjp.net downloading
CMIP6.CMIP.NCC.NorESM2-LM.piControl.r1i1p1f1.Amon.psl.gn.v20210118|esgf-data1.llnl.gov downloading
CMIP6.CMIP.NCC.NorESM2-LM.piControl.r1i1p1f1.Amon.psl.gn.v20210118|esgf-data3.ceda.ac.uk downloading


True

In [4]:
esgfClient.esgfDownloader('GFDL-ESM4','ts_Amon', 'piControl','r1i1p1f1')

GFDL-ESM4ts_AmonpiControlr1i1p1f1 file not found on ESGF


True

In [ ]:
esgfClient.esgfDownloader('EC-Earth3-CC','ts_Amon', 'ssp585','r1i1p1f1')

CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-CC.ssp585.r1i1p1f1.Amon.ts.gr.v20210113|esgf3.dkrz.de downloading


In [15]:
fh.loadModelData('GISS-E2-1-G', 'ts_Amon', 'piControl', 'r1i1p1f2')

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 90, lon: 144, time: 4140)
Coordinates:
  * time       (time) object 7550-01-16 12:00:00 ... 7894-12-16 12:00:00
  * lat        (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
  * lon        (lon) float64 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(612, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(612, 90, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(612, 144, 2), meta=np.ndarray>
    ts         (time, lat, lon) float32 dask.array<chunksize=(612, 90, 144), meta=np.ndarray>
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                Kenneth Lo (cdkkl@giss.nasa.gov)
    ...                     ...
    title:                  GISS-E2-1-G output prepared for CMIP6
    tracking_id:            hdl:21.14100/7c16d62e-003f-46ca-9479-f4c6e596dda0
    variable_id:            ts
    variant_label:          r1i1p1f2
    license:                CMIP6 model data produced by NASA Goddard Institu...
    cmor_version:           3.3.2

# Prototyping for reference / sandbox

In [12]:
from pyesgf.search import SearchConnection

#conn = SearchConnection('https://esgf-node.llnl.gov/esg-search')
#conn = SearchConnection('https://esgf-data.dkrz.de/esg-search')
conn = SearchConnection('https://esgf.nci.org.au/esg-search')

ctx = conn.new_context(
    project='CMIP6', 
    source_id="GFDL-ESM4", 
    experiment_id='piControl', 
    variable='ts_Amon'.split('_')[0], 
    frequency='mon', 
    variant_label='r1i1p1f1',
    #data_node='esgf-data1.llnl.gov'
)
ctx.hit_count

0

In [11]:
results = ctx.search()
[result.dataset_id for result in results]

[]

In [12]:
len(results)

1

In [7]:
result[0].urls

NameError: name 'result' is not defined

In [7]:
#%%writefile dl.sh
result[0].file_context().get_download_script()

'#!/bin/bash\n##############################################################################\n# ESG Federation download script\n#\n# Template version: 1.2\n# Generated by esgf-data.dkrz.de - 2021/06/22 07:51:06\n# Search URL: https://esgf-data.dkrz.de/esg-search/wget?distrib=true&dataset_id=CMIP6.CMIP.CMCC.CMCC-ESM2.piControl.r1i1p1f1.Amon.ts.gn.v20210304|esgf-data1.llnl.gov\n#\n###############################################################################\n# first be sure it\'s bash... anything out of bash or sh will break\n# and the test will assure we are not using sh instead of bash\nif [ $BASH ] && [ `basename $BASH` != bash ]; then\n    echo "######## This is a bash script! ##############" \n    echo "Change the execution bit \'chmod u+x $0\' or start with \'bash $0\' instead of sh."\n    echo "Trying to recover automatically..."\n    sleep 1\n    /bin/bash $0 $@\n    exit $?\nfi\n\nversion=1.3.2\nCACHE_FILE=.$(basename $0).status\nopenId=\nsearch_url=\'https://esgf-data.dkrz.de

In [ ]:
import subprocess
subprocess.check_output('bash dl.sh')

In [ ]:
from pyesgf.logon import LogonManager
lm = LogonManager()
lm.logoff()
lm.is_logged_on()

In [ ]:
myproxy_host = 'esgf-data.dkrz.de'
lm.logon(username=None, password=None, hostname=myproxy_host)
lm.is_logged_on()